In [69]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

## Notebook Summary:

Initial models not really working. only really deep decision trees getting any results, everything else predicting 0s for everything

Did encoding before splitting which I dont really love, not sure what's the proper way to go about this. Also using OrdinalEncoder prolly not right, esp for the name. maybe season can be ordinally encoded.

In [33]:
df = pd.read_csv("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\data\\labelled_bird_sample.csv")

print(df.shape)

df.head()

(103992, 15)


,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common


In [35]:
df.seas_reg_rare.value_counts(normalize=True)

Common      0.654406
Uncommon    0.304495
Rare        0.041099
Name: seas_reg_rare, dtype: float64

**Majority Class Baseline**

65% accuracy

In [68]:
import category_encoders as ce

ModuleNotFoundError: No module named 'category_encoders'

In [43]:
data = df[['name', 'season', 'RegionName', 'seas_reg_rare']]

data.shape

(103992, 4)

In [44]:
encoder = OrdinalEncoder()

data_encoded = encoder.fit_transform(data)

data_encoded.shape

(103992, 4)

In [87]:
train, test = train_test_split(data_encoded, test_size=0.2)

train.shape, test.shape

((83193, 4), (20799, 4))

In [88]:
X_train = train[:, :-1]
y_train = train[:, -1]
X_test = test[:, :-1]
y_test = test[:, -1]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83193, 3), (83193,), (20799, 3), (20799,))

In [89]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500)

model.fit(X_train, y_train)

preds = model.predict(X_test)

print("Accuracy of simple model: ", model.score(X_test, y_test))

Accuracy of simple model:  0.6583489590845714


In [91]:
test = pd.DataFrame({"name": X_test[:, 0],
                     "season": X_test[:, 1],
                     "region": X_test[:, 2],
                     "true_label": y_test,
                     "pred_label": preds})

test

,name,season,region,true_label,pred_label
0,0.636248,0.000000,1.000000,0.0,0.0
1,0.006337,0.666667,0.000000,1.0,0.0
2,0.182510,0.333333,0.153846,0.0,0.0
3,0.182510,0.333333,0.076923,0.0,0.0
4,0.456274,0.666667,0.615385,2.0,0.0
...,...,...,...,...,...
20794,0.558935,0.333333,0.923077,2.0,0.0
20795,0.029151,1.000000,0.000000,0.0,0.0
20796,0.736375,0.000000,0.769231,2.0,0.0
20797,0.871990,1.000000,0.538462,2.0,0.0


In [92]:
test.pred_label.value_counts()

0.0    20799
Name: pred_label, dtype: int64

### Logistic Regression Results

- Logistic regression score 65%, fairing very poorly as that doesn't beat the baseline
- Even after scaling, the results did not improve

In [93]:
from sklearn.tree import DecisionTreeClassifier

In [94]:
train, test = train_test_split(data_encoded, test_size=0.2)

train.shape, test.shape

((83193, 4), (20799, 4))

In [95]:
X_train = train[:, :-1]
y_train = train[:, -1]
X_test = test[:, :-1]
y_test = test[:, -1]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83193, 3), (83193,), (20799, 3), (20799,))

In [106]:
for i in range(5,51,5):
    tree = DecisionTreeClassifier(max_depth=i)

    tree.fit(X_train, y_train)

#     preds = tree.predict(X_test)

    print(f"Tree model (max_depth:{i}) score:", tree.score(X_test, y_test))

Tree model (max_depth:5) score: 0.6758978797057551
Tree model (max_depth:10) score: 0.7548439828837925
Tree model (max_depth:15) score: 0.8567238809558152
Tree model (max_depth:20) score: 0.9325448338862445
Tree model (max_depth:25) score: 0.9666810904370402
Tree model (max_depth:30) score: 0.976921967402279
Tree model (max_depth:35) score: 0.9795182460695225
Tree model (max_depth:40) score: 0.9792778498966297
Tree model (max_depth:45) score: 0.979470166834944
Tree model (max_depth:50) score: 0.9789412952545795


### Tree model 
- Accuracy goes up and up as I increase depth, feels suspiciously overfitt-ey
- Maxes out at around max_depth:30

In [107]:
from sklearn.ensemble import RandomForestClassifier

In [111]:
forest = RandomForestClassifier(n_estimators=1000, max_depth=5)

forest.fit(X_train, y_train)

print("Random Forest Score:", forest.score(X_test, y_test))

Random Forest Score: 0.659214385306986


### Random Forest 

- Similar terrible score, without any hyper-parameter tuning


### Below I try adding more features to the model

In [112]:
df

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103987,Inca Dove,1,United States,Oklahoma,Murray,34.426782,-97.149598,1983-06-03,6,Summer,MurrayOklahoma,Oklahoma,Murray,Appohzarka,Rare
103988,Eastern Phoebe,1,United States,Oklahoma,Murray,34.426782,-97.149598,1994-05-03,5,Spring,MurrayOklahoma,Oklahoma,Murray,Appohzarka,Common
103989,Wood Stork,1,United States,Georgia,Long,31.667925,-81.845470,1997-05-02,5,Spring,LongGeorgia,Georgia,Long,Deep South,Uncommon
103990,Eastern Towhee,1,United States,Tennessee,Giles,35.116398,-87.208443,1987-05-09,5,Spring,GilesTennessee,Tennessee,Giles,Appohzarka,Common


In [113]:
data = df[['name', 'latitude', 'longitude', 'month', 'season', 'RegionName']]

data

,name,latitude,longitude,month,season,RegionName
0,Magnolia Warbler,41.775629,-87.583273,8,Summer,Great Lakes
1,Great Blue Heron,41.956746,-87.913027,7,Summer,Great Lakes
2,Purple Martin,41.707907,-87.886848,6,Summer,Great Lakes
3,Common Yellowthroat,41.707907,-87.886848,6,Summer,Great Lakes
4,Eastern Towhee,41.707907,-87.886848,6,Summer,Great Lakes
...,...,...,...,...,...,...
103987,Inca Dove,34.426782,-97.149598,6,Summer,Appohzarka
103988,Eastern Phoebe,34.426782,-97.149598,5,Spring,Appohzarka
103989,Wood Stork,31.667925,-81.845470,5,Spring,Deep South
103990,Eastern Towhee,35.116398,-87.208443,5,Spring,Appohzarka
